# Treinamento da Rede Neural Classificadora:

In [ ]:
# Instalação de pacotes necessários

%pip install numpy==2.2.6
%pip install matplotlib==3.10.3
%pip install torch
%pip install pandas==2.3.1
%pip install scikit-learn==1.7.1
%pip install seaborn==0.13.2

  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
Using cached nvidia_cublas_

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 53.7 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sklearn-compat 0.1.3 requires scikit-learn<1.7,>=1.2, but you have scikit-learn 1.7.1 which is incompatible.
^C


# New Section

Importando métodos necessários

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import joblib


In [ ]:
df = pd.read_csv('./dataset_fca.csv')

In [ ]:
X = df[['H1', 'H2', 'H3', 'H4', 'H5']].values
y = df['class'].values

In [ ]:
Xp = pd.DataFrame(X);
print(Xp.head())

Codifica a classe de 'R' 'G' para '1','2' etc

In [ ]:
y = df['class'].values

# Converte rótulos de string para números
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

In [ ]:
print(encoder.classes_) #pra ver como codificou

['C' 'G' 'R']


In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
### Normalização, acho que nem precisava porque os dados são todos da mesma feature

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_encoded, test_size=0.7, random_state=42
)


In [ ]:
model = MLPClassifier(
    hidden_layer_sizes=(10),   # ← Camadas ocultas (Sabemos que 10 é o total na rede, talvez alterar pra 5, 5)
    activation='relu',            # ← Função de ativação # talvez alterar para sigmoide
    solver='adam',                # ← Algoritmo de otimização
    max_iter=500,                 # ← Número máximo de épocas (iterações)
    random_state=42               # ← Semente para reprodução de resultados
)



Inicio do teste

In [ ]:
model.fit(X_train, y_train)


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


,hidden_layer_sizes,10
,activation,'relu'
,solver,'adam'
,alpha,0.0001
,batch_size,'auto'
,learning_rate,'constant'
,learning_rate_init,0.001
,power_t,0.5
,max_iter,500
,shuffle,True
,random_state,42


In [ ]:
y_pred = model.predict(X_test)

print("Relatório de Classificação:")
print(classification_report(y_test, y_pred, target_names=encoder.classes_)) #Ver relatorio de classificação, ver com o Cizé como verificar a precisão

Relatório de Classificação:
              precision    recall  f1-score   support

           C       1.00      0.71      0.83        17
           G       0.97      1.00      0.99        38
           R       0.89      1.00      0.94        34

    accuracy                           0.94        89
   macro avg       0.96      0.90      0.92        89
weighted avg       0.95      0.94      0.94        89



In [ ]:
import joblib

In [ ]:
joblib.dump(model, './mlp_model_HB.pkl')
joblib.dump(scaler, './scaler_HB.pkl')
joblib.dump(encoder, './encoder_HB.pkl')


['./encoder_HB.pkl']

# Testar a rede Gerada


## Importar libraries

In [ ]:
import pandas as pd
import joblib


## Importar dataset para teste


In [ ]:
test_df = pd.read_csv('./TestSamples/test_fca.csv')


## Carregar o modelo gerado


In [ ]:
model = joblib.load('./mlp_model_HB.pkl')
scaler = joblib.load('./scaler_HB.pkl')
encoder = joblib.load('./encoder_HB.pkl')

## Setar o modelo de teste com a classe de saida, importante caso o dataset esteja com as colunas alteradas

In [ ]:
X_new = test_df[['H1', 'H2', 'H3', 'H4', 'H5', 'G1', 'G2', 'G3', 'G4', 'G5']].values
y_true = test_df['class'].values  # se você tiver os rótulos verdadeiros no novo dataset

X_scaled = scaler.transform(X_new)


## Testa o modelo com novo dataset


In [ ]:
y_pred_int = model.predict(X_scaled)           # Saída numérica (ex: 0, 1, 2)
y_pred = encoder.inverse_transform(y_pred_int) # Traduz para letras (ex: 'R', 'G', 'C')


### Comparar

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred))


              precision    recall  f1-score   support

           C       0.67      0.67      0.67         3
           G       1.00      0.67      0.80         3
           R       0.75      1.00      0.86         3

    accuracy                           0.78         9
   macro avg       0.81      0.78      0.77         9
weighted avg       0.81      0.78      0.77         9

